<a href="https://colab.research.google.com/github/afortuny/DeepLearningProduction/blob/main/Image_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#hide 
!pip install -Uqq fastbook # remember to restart runtime every time you run this line
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.0 MB/s eta 0:00:00
Mounted at /content/gdrive


In [3]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [4]:
! pip install simple_image_download

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from simple_image_download import simple_image_download as simp



In [6]:
from simple_image_download import simple_image_download as simp 
response = simp.simple_image_download

In [7]:
lst=["adidas running shoes"]

In [8]:
for rep in lst:
 response().download(rep , 300)

In [9]:
dls = ImageDataLoaders.from_folder(Path('/content/simple_images'),valid_pct=0.2)

In [26]:
fns = get_image_files(Path('/content/simple_images'))

In [27]:

failed = verify_images(fns)

In [28]:
failed

(#6) [Path('/content/simple_images/adidas running shoes/adidas running shoes_226.jpg'),Path('/content/simple_images/adidas running shoes/adidas running shoes_170.jpg'),Path('/content/simple_images/adidas running shoes/adidas running shoes_44.jpg'),Path('/content/simple_images/adidas running shoes/adidas running shoes_261.jpg'),Path('/content/simple_images/adidas running shoes/adidas running shoes_79.jpg'),Path('/content/simple_images/adidas running shoes/adidas running shoes_135.jpg')]

In [29]:
path = Path('/content/simple_images')

In [30]:

failed.map(Path.unlink);

In [10]:
dls.valid_ds.items[:3]

[Path('/content/simple_images/adidas running shoes/adidas running shoes_63.jpg'),
 Path('/content/simple_images/adidas running shoes/adidas running shoes_292.jpg'),
 Path('/content/simple_images/adidas running shoes/adidas running shoes_246.jpg')]

In [31]:
def get_dls(size, bs, workers=None):
    source = path
    
    def dummy_label(o): return 0
    files = get_image_files(source)
    tfms = [[PILImage.create, ToTensor, RandomResizedCrop(size, min_scale=0.9)], [dummy_label]]
    
    dsets = Datasets(files, tfms=tfms, splits=RandomSplitter(valid_pct=0.1)(files))
    
    batch_tfms = [IntToFloatTensor]
    dls = dsets.dataloaders(bs=bs, num_workers=workers, after_batch=batch_tfms, device='cpu')
    return dls

In [32]:
bs,resize,size = 64,160,224
dls = get_dls(resize, bs)

In [ ]:
!pip install self-supervised

In [16]:
from self_supervised.layers import *

In [17]:
from self_supervised.vision.moco import *

In [34]:
encoder = create_encoder("tf_efficientnet_b4_ns", n_in=3, pretrained=False) # a timm encoder
model = create_moco_model(encoder, hidden_size=2048, projection_size=128)
aug_pipelines = get_moco_aug_pipelines(size=size)
learn = Learner(dls, model,loss_func=noop,cbs=[MOCO(aug_pipelines=aug_pipelines, K=128)])
learn.fit_flat_cos(100, 1e-2)

epoch,train_loss,valid_loss,time
0,6.061438,7.170436,00:57
1,5.537937,5.240248,01:07
2,5.422298,5.863581,01:07
3,5.330189,6.142441,01:07
4,5.256491,5.991134,01:09
5,5.193027,5.754157,01:07
6,5.158706,4.361912,01:07
7,5.129755,5.392679,01:11
8,5.109294,4.889399,01:09
9,5.098613,5.494907,01:07


/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f6871d43820>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.

epoch,train_loss,valid_loss,time
0,6.061438,7.170436,00:57
1,5.537937,5.240248,01:07
2,5.422298,5.863581,01:07
3,5.330189,6.142441,01:07
4,5.256491,5.991134,01:09
5,5.193027,5.754157,01:07
6,5.158706,4.361912,01:07
7,5.129755,5.392679,01:11
8,5.109294,4.889399,01:09
9,5.098613,5.494907,01:07


/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should b

In [35]:
class SaveFeatures():
    features=None
    def __init__(self, m): 
        self.hook = m.register_forward_hook(self.hook_fn)
        self.features = None
    def hook_fn(self, module, input, output): 
        out = output.detach().cpu().numpy()
        if isinstance(self.features, type(None)):
            self.features = out
        else:
            self.features = np.row_stack((self.features, out))
    def remove(self): 
        self.hook.remove()

In [36]:
learn.model

MoCoModel(
  (encoder): EfficientNet(
    (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNormAct2d(
      48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn1): BatchNormAct2d(
            48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride

In [37]:
learn.save('model.pkl')

Path('models/model.pkl.pth')

In [ ]:
def split_func(m): return L(m[0], m[1]).map(params)

def create_learner(size=size, arch='xresnet34', encoder_path="models/moco_iwang_sz128_epc100_encoder.pth"):
    
    dls = get_dls(size, bs=bs//2)
    pretrained_encoder = torch.load(encoder_path)
    encoder = create_encoder(arch, pretrained=False, n_in=3)
    encoder.load_state_dict(pretrained_encoder)
    nf = encoder(torch.randn(2,3,224,224)).size(-1)
    classifier = create_cls_module(nf, dls.c)
    model = nn.Sequential(encoder, classifier)
    learn = Learner(dls, model, opt_func=opt_func, splitter=split_func,
                metrics=[accuracy,top_k_accuracy], loss_func=LabelSmoothingCrossEntropy())
    return learn

In [39]:
# Second last layer of the model
learn.model

TypeError: ignored

In [ ]:
sf = SaveFeatures(learn.model[1][4])

In [ ]:
_= learn.get_preds(data_big.train_ds)
_= learn.get_preds(DatasetType.Valid)

In [ ]:
len(sf.features)

In [24]:
new_dl = dls.test_dl(dls.valid_ds.items[:], with_labels=True)

In [23]:
valid_lst[1]

Path('/content/simple_images/adidas running shoes/adidas running shoes_246.jpg')

In [ ]:
verify_images()

In [19]:
from google.colab import files
files.download('/content/simple_images/adidas running shoes')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>